# Q# notebooks Sanity test #

This notebook tests all IQ# features.


## Compiling Q# operations

An operation is the basic unit of quantum execution in Q#. It is roughly equivalent to a function in C or C++ or Python, or a static method in C# or Java.

IQ#, the Q# Jupyter kernel, allows you to write Q# operations directly on a code cell and compile them by running the cell (pressing  Ctrl/⌘+Enter). For example:

In [ ]:
operation HelloQ(name: String) : Unit {

    // The following line will simply write a message to the console:
    Message($"Hello {name}!");

}

When you **Run** the cell, Q# compiles the code and returns the name of the operations it found. 
In this case, it found only one operation (`HelloQ`). 

If the compiler detects any errors, it will instead show the list of errors in the output. For example:

Q# operations can call other operations previously defined; they can also call all the operations defined in the 
[Microsoft.Quantum.Primitive](https://docs.microsoft.com/en-us/qsharp/api/prelude/microsoft.quantum.primitive) and 
[Microsoft.Quantum.Canon](https://docs.microsoft.com/en-us/qsharp/api/canon/microsoft.quantum.canon) namespaces (like [Message](https://docs.microsoft.com/en-us/qsharp/api/prelude/microsoft.quantum.primitive.message)).

For example, you can create a new operation that calls the previously compiled `HelloQ`:


In [ ]:
operation HelloAndres() : Unit {
    HelloQ("Andres");
}

They can also use all [Q# Prelude](https://docs.microsoft.com/en-us/qsharp/api/?view=qsharp-preview) operations defined in other namespaces by importing the namespace using the `open` statement. For example, to use [`PI`](https://docs.microsoft.com/en-us/qsharp/api/prelude/microsoft.quantum.extensions.math.pi?view=qsharp-preview) you would need to open the ` Microsoft.Quantum.Extensions.Math` namespace; optionally you can call the operation providing its fully qualified name, for example:
 

In [ ]:
open Microsoft.Quantum.Math;

operation HelloPi() : Unit {
    let pi = Microsoft.Quantum.Convert.DoubleAsString(PI());
    HelloQ(pi);
}

You can define multiple operations in a single cell and use any valid Q# code, for example:

In [ ]:
open Microsoft.Quantum.Math;

/// # Summary
/// Sets the qubit's state to |+⟩
operation SetPlus(q: Qubit) : Unit {
    Reset(q);
    H(q);
}

/// # Summary
/// Sets the qubit's state to |-⟩
operation SetMinus(q: Qubit) : Unit {
    Reset(q);
    X(q);
    H(q);
}

/// # Summary
/// Randomly prepares the qubit into |+⟩ or |-⟩
operation PrepareRandomMessage(q: Qubit) : Unit {

    let choice = RandomInt(2);

    if (choice == 0) {
        Message("Prepared |-⟩");
        SetMinus(q);
    } else {
        Message("Prepared |+⟩");
        SetPlus(q);
    }
}

and create other operations that uses them:

In [ ]:
open Microsoft.Quantum.Diagnostics;

operation TestPrepareQubits() : Result {
    mutable r = Zero;
    
    using (qubits = Qubit[5]) {
        ApplyToEach(PrepareRandomMessage, qubits);
        DumpMachine();
        
        set r = Measure([PauliX, PauliX, PauliX, PauliX, PauliX], qubits);
        
        ResetAll(qubits);
    }
    
    return r;
}

## Simulating Q# operations


Once a Q# operation has been successfully compiled, you can use the `%simulate` command to simulate it. For example:

In [ ]:
%simulate HelloPi

`%simulate` will print any console output on the notebook, and it will return the operation's return value. If the operation returns `Unit` it prints `()`, otherwise it prints the actual value.

`%simulate` only accepts operations that take no arguments. If you want to call an operation that accepts parameters, like `HelloQ`, create a wrapper operation that calls it with the corresponding values, like `HelloPi`.

As mentioned, all messages are printed on the notebook; this includes calls to [`DumpMachine`](https://docs.microsoft.com/en-us/qsharp/api/prelude/microsoft.quantum.extensions.diagnostics.dumpmachine):

In [ ]:
%simulate TestPrepareQubits

## Estimating resources

The `%estimate` command lets you estimate the resources a given quantum operation will need to execute, without actually executing the operation. Similar to `%simulate` it takes the name of a no-arguments operation. However, `%estimate` does not keep track of the qubit's state and will not return the output of the operation, instead it returns the estimated values of how many resources, like Qubits and CNOT gates, the corresponding operation will use:

In [ ]:
%estimate TestPrepareQubits

To learn more about resources estimation, take a look at [The ResourcesEstimator Target Machine](https://docs.microsoft.com/en-us/quantum/machines/resources-estimator?) documentation.

## The Workspace

The notebook uses the folder it lives on disk to define a workspace. It will try to compile all the Q# files (i.e. all files with a `.qs` extension) it finds under the current folder and will make the operations it finds available to operations in the notebook. For example, the [Operations.qs](/edit/Operations.qs) file in this folder defines two operations:
* Microsoft.Quantum.SanityTests.IsMinus
* Microsoft.Quantum.SanityTests.IsPlus

To get the list of operations defined in the workspace, you can use the `%workspace` command:

In [ ]:
%workspace

These operations can be used in this notebook, for example:

In [ ]:
open Microsoft.Quantum.SanityTests;

operation CheckPlus() : Bool {
    mutable result = false;
    
    using (q = Qubit()) {
        SetPlus(q);
        set result = IsPlus(q);
        
        Reset(q);
    }

    return result;
}

In [ ]:
%simulate CheckPlus

To pick up any changes you make to a Q# file in the workspace, use `%workspace reload`. 

In [ ]:
%workspace reload

## Getting Help ##

Q# supports adding documentation to operations via comments in the code. When such documentation exists, you can access it from the notebook by adding a question mark before or after the operation name on a code cell, for example:

In [ ]:
Microsoft.Quantum.Primitive.X?

This documentation is available for any operations in the Prelude, Canon and Workspace, or even those defined locally in the notebook:

In [ ]:
PrepareRandomMessage?

## Other commands ##

### `%who`

`%who` returns the list of all local and workspace operations available.

In [ ]:
%who

### `%package`

`%package` allows you to load nuget packages and makes available any Q# operations defined on them. For example, to use the operations from [Q#'s Quantum Chemistry Library](https://docs.microsoft.com/en-us/quantum/libraries/chemistry/?view=qsharp-preview), you must load the [Microsoft.Quantum.Chemistry](https://www.nuget.org/packages/Microsoft.Quantum.Chemistry/) nuget package:

In [ ]:
%package Microsoft.Quantum.Chemistry::0.11.2004.2824-beta

`%package` returns the list of nuget packages currently loaded and their version.

### `%version`

`%version` simply returns the current versions of IQ# and of Jupyter Core (a library used by IQ#):